In [20]:
# !pip install numpy==1.17.0
# !pip install pandas==0.22.0

# ID2214/FID3214 Assignment 2 Group no. 2
### Project members: 
[Gengcong Yan, gengcong@kth.se]
[Ming Jiang, mingj@kth.se]
[Sihan Chen, sihanc@kth.se]

### Declaration:
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas and time may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas and time

In [21]:
import numpy as np
import pandas as pd
import time

## Reused functions from Assignment 1

In [22]:
# Copy and paste functions from Assignment 1 here that you need for this assignment

# filter away the columns with only na value or only one unique value
def create_column_filter(df):
    # use dropna, how='all' remove the columns with all NaN values
    filtered_df = df.dropna(axis=1, how='all')
    
    # use apply to cal the unique values in columns and find columns where this value==1
    nunique = df.apply(pd.Series.nunique)
    same_value_cols = nunique[nunique == 1].index
    filtered_df = filtered_df.drop(same_value_cols, axis=1)
    
    column_filter = filtered_df.columns.tolist()
    return filtered_df, column_filter

def apply_column_filter(df, column_filter):
    return df[column_filter]

def create_imputation(df):
    imputation = dict()
    for i in df.columns:
        if i != "CLASS" and i != "ID":
            if len(df[i].dropna()) == 0:
                if df[i].dtype == "float64" or df[i].dtype == "int64":
                    imputation[i] = 0
                elif df[i].dtype == "object":
                    imputation[i] = ""
                else: # df[i].dtype == "category":
                    imputation[i] = df[i].cat.categories[0]
            elif df[i].dtype == "float64" or df[i].dtype == "int64":
                imputation[i] = df[i].mean()
            else: # df[i].dtype == "object" or df[i].dtype == "category"
                imputation[i] = df[i].mode().iloc[0]  # multiple modes can exist, mode() should return a dataframe
    df = df.fillna(value=imputation)
    return df, imputation

def apply_imputation(df, imputation):
    df = df.fillna(value=imputation)
    return df

def create_normalization(df, normalizationtype="minmax"):
    normalization = {}
    if normalizationtype=="minmax":        
        for i in df.columns:
            if i != "CLASS" and i != "ID" and (df[i].dtype == "int64" or df[i].dtype == "float64"):       
                temp_min = df[i].min()
                temp_max = df[i].max()
                df[i] = [(x-temp_min)/(temp_max-temp_min) for x in df[i]]
                normalization[i] = (normalizationtype, temp_min, temp_max)
    else: # normalizationtype=="zscore"
        for i in df.columns:
            if i != "CLASS" and i != "ID" and (df[i].dtype == "int64" or df[i].dtype == "float64"):
                temp_mean = df[i].mean()
                temp_std = df[i].std()
                df[i] = [(x-temp_mean)/std for x in df[i]]
                normalization[i] = (normalizationtype, temp_mean, temp_std)
    return df, normalization

def apply_normalization(df, normalization):
    for i in normalization:
        df[i] = [(x-normalization[i][1]) / (normalization[i][2]-normalization[i][1]) for x in df[i]]
    return df

from pandas.api.types import is_numeric_dtype
def create_one_hot(df):
    one_hot = {}
    for col in df.columns:
        if col != 'ID' and col != 'CLASS':
            if not is_numeric_dtype(df[col]):
                df[col] = df[col].astype("category")
                one_hot[col] = list(df[col].cat.categories)
                for i in one_hot[col]:
                    new_col = df[col]==i
                    new_col = new_col.astype("float")
                    df[col + "_" + i] = new_col 
                df = df.drop(columns = col, axis = 1)
    return df, one_hot

def apply_one_hot(df,one_hot):
    for col in df.columns:
        if col in one_hot.keys():    
            for i in one_hot[col]:
                new_col = df[col]==i
                new_col = pd.Series(new_col.astype("float"))
                df[col + "_" + i] = new_col
            df = df.drop(columns = col, axis = 1)
            
    return df   

import math
def folds(df, nofolds=10):
    folds = []
    index = np.random.permutation(df.index)
    num = math.ceil(len(index)/nofolds)
    for i in range(nofolds):
        folds.append(df.loc[index[num*i: num*(i+1)]])
        
    return folds

def accuracy(df, correctlabels):
    true_cases = 0
    all_cases = 0
    for i in df.index:
        all_cases += 1
        max_index = df.iloc[i].idxmax()
        if max_index == correctlabels[i]:
            true_cases += 1
    return true_cases / all_cases

# def brier_score(df, correctlabels):
#     bs_sum = 0
#     bs_N = 0
#     for i in df.index: # 0, 1, 2, 3, 4
#         for j in df.columns: # A, B, C
#             if correctlabels[i]==j:
#                 bs_sum += (df.ix[i][j]-1)**2
#             else:
#                 bs_sum += (df.ix[i][j]-0)**2
#         bs_N += 1
    
#     brier_score = bs_sum / bs_N
#     return brier_score

def brier_score(df, correctlabels):
    df = df.copy()
    brier_score = 0
    for i in df.index:
        p = df.iloc[i]
        o_index = np.where(df.columns==correctlabels[i])[0]
        p.iloc[o_index] -= 1
        brier_score += np.dot(p, p)
    brier_score = brier_score / len(correctlabels)
    return brier_score

def auc(df,correctlabels):
    correctlabels = correctlabels.tolist()
    new_df=df.copy()
    lens=len(correctlabels)
    cols=new_df.columns.tolist()
    if len(cols)<3: #binary ;only Pos and Neg
        pos=[ i for i in range(lens)  if correctlabels[i]==cols[0]]
        neg=[i for i in range(lens)  if correctlabels[i]!=cols[0]]                    
#         print(pos,neg)
        one=new_df[cols[0]]
        auc = 0
        for i in pos:
            for j in neg:
                if one[i] > one[j]:
                    auc += 1
                elif one[i] == one[j]:
                    auc += 0.5

        return auc / (len(pos)*len(neg))
    else:      # CLASS >=3   Pos / non-Pos
        aucs=[]
        for col in cols:
            pos=[ i for i in range(lens)  if correctlabels[i]==col]
            nonpos=[i for i in range(lens)  if correctlabels[i]!=col]                      
#             print(pos,nonpos)
            one=new_df[col]
            auc = 0
            for i in pos:
                for j in nonpos:
                    if one[i] > one[j]:
                        auc += 1
                    elif one[i] == one[j]:
                        auc += 0.5
            auc=auc/(len(pos)*len(nonpos))
            aucs.append(auc)
        weights=[correctlabels.count(col)/len(correctlabels) for col in cols ]
#         print(weights)
#         print(aucs)

        weights=np.array(weights)
        aucs=np.array(aucs)
        avg_auc=np.sum(aucs*weights)
        return avg_auc

    
def create_bins(df, nobins=10, bintype="equal-width"):
    binning = dict()
    if bintype=="equal-width":
        for i in df.columns:
            if i != "CLASS" and i != "ID" and (df[i].dtype == "float64" or df[i].dtype == "int64"):
                res, bins = pd.cut(df[i], nobins, retbins=True, labels=False)  # labels=False only display which bin the value belongs to
                df[i] = res.astype("category") # Hint 4,5
                bins[0], bins[-1] = -np.inf, np.inf
                binning[i] = bins
    else: # bintype=="equal-size"
        for i in df.columns:
            if i != "CLASS" and i != "ID" and (df[i].dtype == "float64" or df[i].dtype == "int64"): # bin edges must be unique
                res, bins = pd.qcut(df[i], nobins, retbins=True, labels=False, duplicates="drop")
                df[i] = res.astype("category")
                bins[0], bins[-1] = -np.inf, np.inf
                binning[i] = bins
    return df, binning

def apply_bins(df, binning):
    for i in binning:
        res = pd.cut(df[i], bins=binning[i], labels=False)
        df[i] = res.astype("category")
    return df

## 1. Define the class kNN

In [23]:
# Define the class kNN with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, normalization, one_hot, labels, training_labels, training_data, training_time
#
# Input to fit:
# self              - the object itself
# df                - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# normalizationtype - "minmax" (default) or "zscore"
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter   - a column filter (see Assignment 1) from df
# self.imputation      - an imputation mapping (see Assignment 1) from df
# self.normalization   - a normalization mapping (see Assignment 1), using normalizationtype from the imputed df
# self.one_hot         - a one-hot mapping (see Assignment 1)
# self.training_labels - a pandas series corresponding to the "CLASS" column, set to be of type "category" 
# self.labels          - a list of the categories (class labels) of the previous series
# self.training_data   - the values (an ndarray) of the transformed dataframe, i.e., after employing imputation, 
#                        normalization, and possibly one-hot encoding, and also after removing the "CLASS" and "ID" columns
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# k    - an integer >= 1 (default = 5)
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the relative class frequencies in the set of class labels from the k nearest 
#               (with respect to Euclidean distance) neighbors in training_data
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply column filtering, imputation, normalization and one-hot
#
# Hint 2: Get the numerical values (as an ndarray) from the resulting dataframe and iterate over the rows 
#         calling some sub-function, e.g., get_nearest_neighbor_predictions(x_test,k), which for a test row
#         (numerical input feature values) finds the k nearest neighbors and calculate the class probabilities.
#
# Hint 3: This sub-function may first find the distances to all training instances, e.g., pairs consisting of
#         training instance index and distance, and then sort them according to distance, and then (using the indexes
#         of the k closest instances) find the corresponding labels and calculate the relative class frequencies

class kNN:
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.normalization = None
        self.one_hot = None
        self.training_labels = None
        self.labels = None
        self.training_data = None
    
    def fit(self, dataframe, normalizationtype="minmax"):
        df = dataframe.copy()
        df, self.column_filter  = create_column_filter(df)
        df, self.imputation = create_imputation(df)
        df, self.normalization = create_normalization(df, normalizationtype)
        df, self.one_hot = create_one_hot(df)
        df["CLASS"] = df["CLASS"].astype("category")
        self.training_labels = df["CLASS"]
        self.labels = list(df["CLASS"].cat.categories)
        self.training_data = df.drop(columns=["ID", "CLASS"], errors='ignore')
        
#         print("training data:", self.training_data)
    
    def euclidean_distance(self, row1, row2):
        return np.sqrt(np.sum(np.power(row1 - row2, 2), axis=0))
    
    def get_NN_predictions(self, x_test, k):
        neighbours = []
        dists = []
        data = self.training_data
        
        # for every row, calculate the euclidean distance between itself and the testing data row
        for i in range(data.shape[0]):
            dist = 0.0
            values = np.array(data.iloc[i,:].values)
            dist = self.euclidean_distance(values, x_test)
            
            dists.append((i, dist))  # a row, a float number
        
        dists.sort(key=lambda tup: tup[1])  # sort by distance
        
        # return the first k elements (CLASS value) as neighbors
        for j in range(k):
            neighbours.append(dists[j][0])
        
        return neighbours
    
    def get_probability(self, k_NN_indexes):
        labels = self.labels
        k = len(k_NN_indexes)
        labels_prob = np.zeros(len(labels))
        
        for i in range(k):
            for j in range(len(labels)):
                if self.training_labels[k_NN_indexes[i]] == labels[j]:
                    labels_prob[j] += 1
        prob = labels_prob/k
        return prob
    
    def predict(self, dataframe, k):
        df = dataframe.copy()
        df.drop(columns=["ID", "CLASS"], inplace=True)
        
#         df = apply_column_filter(df, self.column_filter)
        df = apply_normalization(df, self.normalization)
        df = apply_imputation(df, self.imputation)
        df = apply_one_hot(df, self.one_hot)
        num_rows = df.shape[0]
        s = (num_rows, len(self.labels))
        predictions = np.zeros(s)
        
        for i in range(num_rows):
            values = np.array(df.iloc[i,:].values)
            neightbours_index = self.get_NN_predictions(values, k)
            prob = self.get_probability(neightbours_index)
            predictions[i] = prob
        
        predictions_df = pd.DataFrame(predictions, columns=self.labels)
        
        return predictions_df

In [24]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

knn_model = kNN()

t0 = time.perf_counter()
knn_model.fit(glass_train_df)
print("Training time: {0:.2f} s.".format(time.perf_counter()-t0))

test_labels = glass_test_df["CLASS"]

k_values = [1,3,5,7,9]
results = np.empty((len(k_values),3))

for i in range(len(k_values)):
    t0 = time.perf_counter()
    predictions = knn_model.predict(glass_test_df,k=k_values[i])
    print("Testing time (k={0}): {1:.2f} s.".format(k_values[i],time.perf_counter()-t0))
#     print("pred", predictions)
#     print("tl",test_labels)
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=k_values,columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time: 0.02 s.
Testing time (k=1): 2.07 s.
Testing time (k=3): 2.10 s.
Testing time (k=5): 2.19 s.
Testing time (k=7): 2.33 s.
Testing time (k=9): 2.41 s.



'results'

,Accuracy,Brier score,AUC
1,0.747664,0.504673,0.810350
3,0.663551,0.488058,0.815859
5,0.579439,0.474019,0.833805
7,0.598131,0.470723,0.834465
9,0.616822,0.483674,0.828734


In [25]:
train_labels = glass_train_df["CLASS"]
predictions = knn_model.predict(glass_train_df,k=1)
print("Accuracy on training set (k=1): {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set (k=1): {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set (k=1): {0:.4f}".format(brier_score(predictions,train_labels)))

Accuracy on training set (k=1): 1.0000
AUC on training set (k=1): 1.0000
Brier score on training set (k=1): 0.0000


### Comment on assumptions, things that do not work properly, etc.


## 2. Define the class NaiveBayes

In [26]:
# Define the class NaiveBayes with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, binning, labels, class_priors, feature_class_value_counts, feature_class_counts
#
# Input to fit:
# self    - the object itself
# df      - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins  - no. of bins (default = 10)
# bintype - either "equal-width" (default) or "equal-size" 
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter              - a column filter (see Assignment 1) from df
# self.binning                    - a discretization mapping (see Assignment 1) from df
# self.class_priors               - a mapping (dictionary) from the labels (categories) of the "CLASS" column of df,
#                                   to the relative frequencies of the labels
# self.labels                     - a list of the categories (class labels) of the "CLASS" column of df
# self.feature_class_value_counts - a mapping from the feature (column name) to the number of
#                                   training instances with a specific combination of (non-missing, categorical) 
#                                   value for the feature and class label
# self.feature_class_counts       - a mapping from the feature (column name) to the number of
#                                   training instances with a specific class label and some (non-missing, categorical) 
#                                   value for the feature
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the naive approximation of Bayes rule (see lecture slides)
#
# Hint 1: First apply the column filter and discretization
#
# Hint 2: Iterating over either columns or rows, and for each possible class label, calculate the relative
#         frequency of the observed feature value given the class (using feature_class_value_counts and 
#         feature_class_counts) 
#
# Hint 3: Calculate the non-normalized estimated class probabilities by multiplying the class priors to the
#         product of the relative frequencies
#
# Hint 4: Normalize the probabilities by dividing by the sum of the non-normalized probabilities; in case
#         this sum is zero, then set the probabilities to the class priors

class NaiveBayes:
    def __init__(self):
        self.column_filter =None
        self.binning = None
        self.class_priors = None
        self.feature_class_value_counts = None
        self.feature_class_counts = None
        self.labels = None
    
    def fit(self, dataframe, nobins=10, bintype="equal-width"):
        df = dataframe.copy()
        df, self.column_filter = create_column_filter(df)
        df, self.binning = create_bins(df, nobins, bintype)
        df["CLASS"] = df["CLASS"].astype("category")
        self.labels = list(df["CLASS"].cat.categories)
        self.class_priors = dict(df["CLASS"].value_counts(normalize=True))
        
        # a mapping from a col to a dictionary (c, num of instances)
        feature_class_counts = {}
        # a mapping from a col to a dictionary((c,v),num of this combination)
        feature_class_value_counts = {}

        for col in df.columns:
            if col not in ["CLASS", "ID"]:
                df_temp = df.dropna(axis=0, how="any", subset=["CLASS",col])
                # {(class 1, num of existence)}
                feature_class_counts[col] = dict(df_temp["CLASS"].value_counts())
                g = df_temp.groupby(["CLASS", col]).size()
                # {(class 1, column value bin1): num of existence, (class1, column value bin2): num of existence,...}
                feature_class_value_counts[col] = dict(g)
        self.feature_class_counts = feature_class_counts
        self.feature_class_value_counts = feature_class_value_counts
        
    def predict(self, dataframe):
        df = dataframe.copy()
        df = apply_column_filter(df, self.column_filter)
        df = apply_bins(df, self.binning)
        labels = self.labels # class categories
        df.drop(columns=["ID", "CLASS"], inplace=True)
        
        nrow, ncol, nlabel = df.shape[0], df.shape[1], len(labels)
        matrix = np.zeros([nlabel, nrow, ncol])
        
        #  Iterating over either columns or rows, and for each possible class label, calculate the relative
        #  frequency of the observed feature value given the class (using feature_class_value_counts and 
        #  feature_class_counts) 
        for col_num in range(ncol):
            col = df.columns[col_num]
            
            for label_num in range(nlabel):
                label = labels[label_num]
                
                for row_num in range(nrow):
                    value = df.iloc[row_num, col_num]
                    if((label, value) in self.feature_class_value_counts[col].keys()):
                        features_value_count = self.feature_class_value_counts[col][(label, value)]
                        feature_count = self.feature_class_counts[col][label]
                        relative_freq = features_value_count / feature_count
                    else:
                        relative_freq = 0
                    
                    # normalized probability from the training data
                    matrix[label_num, row_num, col_num] = relative_freq
        

#         Normalize the probabilities by dividing 
#         by the sum of the non-normalized probabilities; in case
#         this sum is zero, then set the probabilities to the class priors
        prior = np.array([self.class_priors[labels[i]] for i in range(nlabel)])
        
        # relative_freq product of likelihood in under one label 
        relative_freq = matrix.prod(axis=2)
        
        priors = np.tile(prior, nrow).reshape(nrow, nlabel)

        relative_freq = relative_freq.T
        class_prob = relative_freq * priors
        predictions = pd.DataFrame(class_prob, columns=labels)
        
        # normalization
        sum_ = predictions.sum(axis=1)# sum of possibilities
        for i in range(len(df)):
            for label in self.labels:
                predictions.loc[i, label] = predictions.loc[i, label]/sum_[i] \
                if sum_[i] != 0 else self.class_priors[label]
        
        return predictions
        

In [27]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

nb_model = NaiveBayes()

test_labels = glass_test_df["CLASS"]

nobins_values = [3,5,10]
bintype_values = ["equal-width","equal-size"]
parameters = [(nobins,bintype) for nobins in nobins_values for bintype in bintype_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    nb_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = nb_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values]),
                       columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time (3, 'equal-width'): 0.07 s.
Testing time (3, 'equal-width'): 0.32 s.
Training time (3, 'equal-size'): 0.07 s.
Testing time (3, 'equal-size'): 0.32 s.
Training time (5, 'equal-width'): 0.07 s.
Testing time (5, 'equal-width'): 0.32 s.
Training time (5, 'equal-size'): 0.08 s.
Testing time (5, 'equal-size'): 0.32 s.
Training time (10, 'equal-width'): 0.07 s.
Testing time (10, 'equal-width'): 0.30 s.
Training time (10, 'equal-size'): 0.08 s.
Testing time (10, 'equal-size'): 0.32 s.



'results'

Accuracy  Brier score       AUC
3  equal-width  0.616822     0.622116  0.724335
   equal-size   0.607477     0.554782  0.780163
5  equal-width  0.644860     0.551101  0.771688
   equal-size   0.598131     0.581556  0.796675
10 equal-width  0.654206     0.527569  0.812887
   equal-size   0.588785     0.741668  0.751165

In [28]:
train_labels = glass_train_df["CLASS"]
nb_model.fit(glass_train_df)
predictions = nb_model.predict(glass_train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.8505
AUC on training set: 0.9687
Brier score on training set: 0.2263


### Comment on assumptions, things that do not work properly, etc.